In [1]:
import nltk

In [2]:
#nltk.download_shell()

In [2]:
messages = [line.rstrip() for line in open('smsspamcollection/SMSSpamCollection')]

In [3]:
len(messages)

5574

In [4]:
messages[0]

'ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [6]:
for mess_no , message in enumerate(messages[:10]):
    print(mess_no,message)
    print('\n')

0 ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...


1 ham	Ok lar... Joking wif u oni...


2 spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


3 ham	U dun say so early hor... U c already then say...


4 ham	Nah I don't think he goes to usf, he lives around here though


5 spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, Â£1.50 to rcv


6 ham	Even my brother is not like to speak with me. They treat me like aids patent.


7 ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune


8 spam	WINNER!! As a valued network customer you have been selected to receivea Â£900 prize reward! To claim call 09061701461. Claim cod

In [5]:
import pandas as pd

In [6]:
df= pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t',names=['labels','messages'])

In [7]:
df.head()

,labels,messages
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df['length']=df['messages'].apply(len)

In [9]:
df.head()

,labels,messages,length
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [10]:
import seaborn as sns

In [11]:
import matplotlib.pyplot as plt

In [12]:
%matplotlib inline

In [15]:
# message['messages'].hist(hue='label')

In [13]:
df.groupby('labels')['messages'].describe()

,count,unique,top,freq
labels,,,,
ham,4825,4516,"Sorry, I'll call later",30
spam,747,653,Please call our customer service representativ...,4


In [14]:
import string

In [15]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [16]:
mess = "this is a , message . with punctuation , and its very dirty!!@"

In [17]:
no_punc = [i  for i in mess if i not in string.punctuation]

In [18]:
no_punc="".join(no_punc)

In [19]:
no_punc=no_punc.split()

In [20]:
from nltk.corpus import stopwords

In [21]:
clean=[c for c in no_punc if c not in stopwords.words('english') ]

In [22]:
clean

['message', 'punctuation', 'dirty']

In [23]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [25]:
df['messages'].head(5).apply(text_process)

0    [Go, jurong, point, crazy, Available, bugis, n...
1                       [Ok, lar, Joking, wif, u, oni]
2    [Free, entry, 2, wkly, comp, win, FA, Cup, fin...
3        [U, dun, say, early, hor, U, c, already, say]
4    [Nah, dont, think, goes, usf, lives, around, t...
Name: messages, dtype: object

In [26]:
len(df['messages'])

5572

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
# Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_process).fit(df['messages'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

11425


In [30]:
message_bow = bow_transformer.transform(df['messages'])

In [31]:
from sklearn.feature_extraction.text import TfidfTransformer

In [34]:
message_bow.nnz

50548

In [37]:
tfid= TfidfTransformer().fit(message_bow)

In [39]:
tfif_tra = tfid.transform(message_bow)

In [40]:
from sklearn.naive_bayes import MultinomialNB

In [41]:
model = MultinomialNB().fit(tfif_tra,df['labels'])

In [43]:
model.predict(tfif_tra)

array(['ham', 'ham', 'spam', ..., 'ham', 'ham', 'ham'], dtype='<U4')

In [45]:
from sklearn.model_selection import train_test_split

In [44]:
from sklearn.pipeline import Pipeline

In [47]:
msg_train,msg_test,y_train,y_test = train_test_split(df['messages'],df['labels'],test_size=0.3)

In [50]:
pip = Pipeline([
    ('bow',CountVectorizer(analyzer = text_process)),
    ('tfid',TfidfTransformer()),
    ('model',MultinomialNB())
])

In [53]:
pip.fit(msg_train,y_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x000002B710A99A68>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfid',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('model',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [55]:
pred=pip.predict(msg_test)

In [56]:
from sklearn.metrics import classification_report

In [57]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      1451
        spam       1.00      0.70      0.82       221

    accuracy                           0.96      1672
   macro avg       0.98      0.85      0.90      1672
weighted avg       0.96      0.96      0.96      1672

